# Problem 10.4 - Heat Equation with Markers

In [1]:
# Import necessary packages
using SparseArrays
using StaticArrays
using LinearAlgebra
using IterativeSolvers
using WriteVTK
using Printf

include("Grid.jl")
include("Markers.jl")
include("Stokes.jl")

compute_timestep (generic function with 1 method)

In this example, we prescribe the velocity field with a constant vx=vy=$10^{-9}$ m/s. When markers exit the domain, they should re-enter the other side.

Energy Equation:
$$
\rho C_p \frac{\partial T}{\partial t} = \nabla\cdot(k \nabla T) + H
$$

Implicit form:
$$
\rho C_p \frac{T^{n+1}-T^n}{\Delta t} = \nabla\cdot(k \nabla T^{n+1}) + H
$$

$$
\rho C_p \frac{T^{n+1}}{\Delta t} - \nabla\cdot(k \nabla T^{n+1}) =  H + \rho C_p \frac{T^n}{\Delta t} 
$$

In [2]:
function move_markers_constant_velocity!(markers::Markers,grid::CartesianGrid,vx,vy,dt)
    for i in 1:markers.nmark
        markers.x[1,i] += dt*vx
        markers.x[2,i] += dt*vy
    end
    replace_markers!(markers,grid)
    find_cells!(markers,grid)
end

function replace_markers!(markers::Markers,grid::CartesianGrid)
     for i in 1:markers.nmark
        if markers.x[1,i] > grid.W
            markers.x[1,i] -= grid.W
        elseif markers.x[1,i] < 0.0
            markers.x[1,i] += grid.W
        end
        if markers.x[2,i] > grid.H
            markers.x[2,i] -= grid.H
        elseif markers.x[2,i] < 0.0
            markers.x[2,i] += grid.H
        end
    end
end

function subgrid_temperature_relaxation!(markers::Markers,grid::CartesianGrid,Tlast::Matrix,Cp,kThermal,dt::Float64)
        dsubgrid = 1.0; # subgrid temperature diffusivity
        dT_subgrid_m = Array{Float64,1}(undef,markers.nmark)
        # compuate the nodal temperature on the markers.
        basic_node_to_markers!(markers,grid,Tlast,dT_subgrid_m)
        # compute the subgrid temperature changes on the markers
        rho = markers.scalarFields["rho"]
        T = markers.scalarFields["T"]
        Threads.@threads for i in 1:markers.nmark
            dx2 = (grid.x[markers.cell[1,i]+1] - grid.x[markers.cell[1,i]])^2
            dy2 = (grid.y[markers.cell[2,i]+1] - grid.y[markers.cell[2,i]])^2
            tdiff = markers.scalars[rho,i]*Cp/kThermal / (2/dx2 + 2/dy2)
            dT_subgrid_m[i] = (dT_subgrid_m[i]-markers.scalars[T,i])*( 1.0 - exp(-dsubgrid*dt/tdiff) )
        end
        # interpolate subgrid temperature changes back onto basic nodes.
        markers.scalars[T,1:markers.nmark] += dT_subgrid_m
    
        dT_subgrid_node = marker_to_basic_node(markers,grid,dT_subgrid_m)
        return dT_subgrid_node
end

# Define a function to form the energy equation left hand side and right hand side
function assemble_energy_equation(grid::CartesianGrid,rho,Cp,kThermal,H,Tlast,dt)
    N = grid.nx*grid.ny
    row = zeros(Int64,5*N);
    col = zeros(Int64,5*N);
    val = zeros(Float64, 5*N);
    R = zeros(Float64,N,1);
    k = 1;
    
    for j in 1:grid.nx
        dxc = grid.x[2] - grid.x[1] # uniform spacing for now...
        dxp = dxc;
        dxm = dxc;
        for i in 1:grid.ny
            dyc = grid.y[2] - grid.y[1] # uniform spacing for now...
            dyp = dyc;
            dym = dyc;
            this_row = node_index(i,j,grid.ny);
            # kA, kB, kC, kD
            kA = j==1 ? kThermal[i,j] : 0.5*(kThermal[i,j-1] + kThermal[i,j])
            kB = j==grid.nx ? kThermal[i,j] : 0.5*(kThermal[i,j] + kThermal[i,j+1])
            kC = i==1 ? kThermal[i,j] : 0.5*(kThermal[i-1,j] + kThermal[i,j])
            kD = i==grid.ny ? kThermal[i,j] : 0.5*(kThermal[i,j] + kThermal[i+1,j])
            # diagonal entry
            row[k] = this_row
            col[k] = this_row
            val[k] = (rho[i,j]*Cp[i,j])/dt + kB/dxp/dxc + kA/dxm/dxc + kD/dyp/dyc + kC/dyp/dyc;
            k+=1
            # right
            row[k] = this_row
            col[k] = j==grid.nx ? node_index(i,1,grid.ny) : node_index(i,j+1,grid.ny);
            val[k] = -kB/dxp/dxc;
            k+=1
            # left
            row[k] = this_row
            col[k] = j==1 ? node_index(i,grid.nx,grid.ny) : node_index(i,j-1,grid.ny);
            val[k] = -kA/dxm/dxc;
            k+=1
            # down (+y)
            row[k] = this_row
            col[k] = i==grid.ny ? node_index(1,grid.nx,grid.ny) : node_index(i+1,j,grid.ny);
            val[k] = -kD/dyp/dyc;
            k+=1
            # up (-y)
            row[k] = this_row
            col[k] = i==1 ? node_index(grid.ny,j,grid.ny) : node_index(i-1,j,grid.ny);
            val[k] = -kC/dyp/dyc;
            k+=1
            R[this_row] = Tlast[i,j]*rho[i,j]*Cp[i,j]/dt            
        end
    end
    row = @views row[1:k-1]
    col = @views col[1:k-1]
    val = @views val[1:k-1]
    L = sparse(row,col,val)
    return L,R
end

assemble_energy_equation (generic function with 1 method)

In [3]:
# Initial conditions for this problem - assign marker density and transport properties.
function initial_conditions!(grid::CartesianGrid,markers::Markers)
    T = markers.scalarFields["T"]
    rho = markers.scalarFields["rho"]
    for i in 1:markers.nmark
        if markers.x[1,i] > 4e5 && markers.x[1,i] < 6e5 && markers.x[2,i] > 6e5 && markers.x[2,i] < 9e5
            markers.scalars[T,i] = 1300.
            markers.scalars[rho,i] = 3200.
        else
            markers.scalars[T,i] = 1000.
            markers.scalars[rho,i] = 3200.
        end
    end
end

function visualization(grid::CartesianGrid,rho::Matrix,eta::Matrix,vn::Array{Float64},pressure::Matrix,temperature::Matrix,time ; filename="test.vts")
    # write the visualization output from the regular grid as a .vts file.
    vtk_grid(filename, grid.x, grid.y) do vtk
        vtk["rho"] = transpose(rho)
        vtk["viscosity"] = transpose(eta)
        # add a fake third dimension to the velocity vectors
        v3 = Array{Float64,3}(undef,3,grid.nx,grid.ny)
        v3[1:2,:,:] = vn
        v3[3,:,:] .= 0.0
        vtk["Velocity"] = v3
        vtk["Temperature"] = transpose(temperature)
        vtk["pressure"] = transpose(pressure[2:end,2:end])
        vtk["TIME"] = time
    end
end

function visualization(markers::Markers,time; filename="markers.vtp")  
    p3 = Array{Float64,2}(undef,3,markers.nmark)
    p3[1:2,:] = markers.x[1:2,1:markers.nmark]
    p3[3,:] .= 0.0
      
    polys = [MeshCell(PolyData.Polys(),i:i) for i in 1:markers.nmark]
    vtk_grid(filename,p3,polys) do vtk    
        for key in keys(markers.scalarFields)
            vtk[key] = markers.scalars[markers.scalarFields[key],1:markers.nmark]
        end
       vtk["TIME"] = time
    end
end

visualization (generic function with 2 methods)

In [5]:
nx = 51
ny = 76
W = 1e6
H = 1.5e6

vx = 1e-9
vy = 1e-9

plot_interval = W/vx/20.

markx = 5
marky = 5
seconds_in_year = 3.15e7
plot_interval = 1e5*seconds_in_year # plot interval in seconds
end_time = W/vx # end time in seconds
dtmax = plot_interval
grid = CartesianGrid(W,H,nx,ny)
println("Creating Markers...")
@time markers = Markers(grid,["T","rho","eta"],[] ; nmx=markx,nmy=marky,random=false)
println("Initial condition...")
@time initial_conditions!(grid,markers)

# define arrays for k, rho, cp, H at the basic nodes. Fill them with constant values for now.
rho = zeros(grid.ny,grid.nx);
kThermal = zeros(grid.ny,grid.nx);
Cp = zeros(grid.ny,grid.nx);
H = zeros(grid.ny,grid.nx);
fill!(rho,3200.);
fill!(kThermal,3.0);
fill!(Cp,1000.);

time = 0.
last_plot=0.
iout=0

while time <= end_time
    # get temperature from last timestep
    fields = marker_to_basic_node(markers,grid,["T","rho"])
    Told = fields[1,:,:]
    rho = fields[2,:,:]
    # determine timestep
    dt = 0.5*min((grid.x[2]-grid.x[1])/vx,(grid.y[2]-grid.y[1])/vy)
    
    L,R = assemble_energy_equation(grid,rho,Cp,kThermal,H,Told,dt)
    Tnew = L\R;
    Tnew = reshape(Tnew,grid.ny,grid.nx);
    # compute the temperature change
    dTemp = Tnew-Told
    # calculate subgrid temperature change.
    dT_subgrid_node = subgrid_temperature_relaxation!(markers,grid,Told,Cp[1,1],kThermal[1,1],dt)
    dT_remaining = dTemp - dT_subgrid_node
    
    basic_node_change_to_markers!(markers,grid,dT_remaining,"T")
    move_markers_constant_velocity!(markers,grid,vx,vy,dt)
    time += dt
    
    if time == 0.0 || time - last_plot >= plot_interval
        last_plot = time 
        name = @sprintf("output_chapter10/viz.%04d.vtr",iout)
        vn = zeros(2,grid.nx,grid.ny)
        vn[1,:,:] .= vx
        vn[2,:,:] .= vy
        eta_s = zeros(grid.ny,grid.nx)
        P = eta_s
        visualization(grid,rho,eta_s,vn,P,Tnew,time/seconds_in_year;filename=name)
        name = @sprintf("output_chapter10/markers.%04d.vtp",iout)
        visualization(markers,time/seconds_in_year;filename=name)
        iout += 1
    end
end


Creating Markers...
  0.002330 seconds (20 allocations: 5.176 MiB)
Initial condition...
  0.002076 seconds


LoadError: InterruptException:

In [15]:
@time    fields = marker_to_basic_node(markers,grid,["T","rho"])

@time    dT_subgrid_node = subgrid_temperature_relaxation!(markers,grid,fields[1,:,:],Cp[1,1],kThermal[1,1],1.0)


  0.426960 seconds (6.09 M allocations: 94.606 MiB, 2.40% gc time)
  0.339272 seconds (7.78 M allocations: 122.424 MiB, 6.46% gc time)


76×51 Matrix{Float64}:
 3.77336e-25  8.95079e-25  4.05193e-24  …  4.4782e-24    1.51575e-24
 5.9897e-25   1.27737e-24  5.1859e-24      5.70512e-24   2.10122e-24
 2.55988e-24  3.83304e-24  1.0971e-23      1.19245e-23   5.3733e-24
 1.38903e-23  1.61561e-23  2.87388e-23     3.03782e-23   1.88766e-23
 7.53683e-23  7.90117e-23  9.92023e-23     1.01754e-22   8.33945e-23
 3.83249e-22  3.88571e-22  4.17997e-22  …  4.21722e-22   3.95002e-22
 1.81185e-21  1.81897e-21  1.85824e-21     1.86328e-21   1.8276e-21
 8.01428e-21  8.02309e-21  8.07158e-21     8.07788e-21   8.03379e-21
 3.3349e-20   3.33592e-20  3.34153e-20     3.34226e-20   3.33716e-20
 1.306e-19    1.30611e-19  1.30672e-19     1.3068e-19    1.30624e-19
 4.79075e-19  4.79087e-19  4.79152e-19  …  4.7916e-19    4.79101e-19
 1.63167e-18  1.63168e-18  1.63175e-18     1.63176e-18   1.6317e-18
 5.09417e-18  5.09418e-18  5.09425e-18     5.09426e-18   5.0942e-18
 ⋮                                      ⋱                ⋮
 4.6438e-27   5.65399e-27

In [33]:
for key in keys(markers.scalarFields)
    println(key)
end

T
rho
